In [1]:
import json
import zipfile
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import math
import csv
import operator
import Geohash
from build import TreeBuilding
from decision import haversine
from sklearn.cross_validation import train_test_split
import pickle
    
%matplotlib inline

In [2]:
air_trajs = pd.read_csv('air_trajs.csv',
                        #index_col=,
                        usecols = ['POLYLINE', 'Destination'],
                        converters={'POLYLINE': lambda x: json.loads(x), 'Destination': lambda x: json.loads(x)})

In [3]:
trajectories = list(air_trajs['POLYLINE'])
targets = list(air_trajs['Destination'])

In [4]:
X_train, X_test, y_train, y_test = train_test_split(trajectories, targets, train_size=0.70, random_state = 0)

In [ ]:
tb = TreeBuilding()
tb = tb.fit(X_train, y_train, 1, 10)

In [ ]:
def prediction_error(y_val, y_hat):
    return np.mean(np.power([haversine(y_val[i], y_hat[i]) for i in xrange(len(y_val))], 2))

In [ ]:
y_hat = tb.predict(tb, X_test)

In [ ]:
prediction_error(y_test, y_hat)

In [ ]:
def mean_points_alpha(points):
    lats = [point[0] for point in points]
    lons = [point[1] for point in points]
    return [np.mean(lats), np.mean(lons)]

def sqare_error_alpha(points):
    pmean = mean_points_alpha(points)
    return np.mean(np.power([haversine(point, pmean) for point in points], 2))

In [ ]:
sqare_error_alpha(y_test)

In [ ]:
print tb.root.decision_point

In [ ]:
pickle.dump(tb, open( \"tree_10_new.p\", \"wb\" ))

In [ ]:
load_tree = pickle.load( open( \"tree_10_new.p\", \"rb\" ))

In [ ]:
y_hat = predict_batch(load_tree, X_test)
prediction_error(y_test, y_hat)

In [ ]:
import gmplot

gmap = gmplot.GoogleMapPlotter(41.1496100, -8.6109900 , 12)

#gmap.plot(heat_lats, heat_lons, 'cornflowerblue', edge_width=10)
#gmap.scatter(heat_lats, heat_lons, '#3B0B39', size=10, marker=False)
gmap.scatter(start_lats, start_lons, '#006400', size=10, marker=False)
gmap.scatter(end_lats, end_lons, '#8B0000', size=10, marker=False)

#gmap.heatmap(heat_lats, heat_lons)

gmap.draw("\mymap.html\")